# Copyright Netherlands eScience Center <br>
** Function     : Migrate ConvLSTM pkl from deepclim package to dlcas package** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.07.29 ** <br>
** Last Update  : 2020.07.29 ** <br>
** Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib **<br>
Description     : This notebook serves to migrate ConvLSTM pkl from deepclim package to dlcas package. <br>

Return Values   : pkl model <br>

In [1]:
%matplotlib inline

import sys
import os
import numpy as np

import torch
import torch.nn.functional

sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
import dlacs
import dlacs.ConvLSTM

sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DeepClim")
# legacy
import deepclim
import deepclim.deepArray

In [2]:
################################################################################# 
#########                           modelpath                             ########
#################################################################################
# path of initialization weight
init_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\Maps'
# path of output
output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\modelInit'

In [3]:
if __name__=="__main__":
    print ('*******************  create basic dimensions for network  *********************')
    # specifications of neural network
    ####################  two variable network   ####################
    # variable list
    # sic var month
    input_channels = 3
    hidden_channels = [3, 3, 2]
    kernel_size = 3
    ####################  all variable network   ####################
    # variable list
    # sic ohc t2m slp z500 z850 u10m v10m sflux month
    #input_channels = 10
    #hidden_channels = [10, 10, 9]
    #kernel_size = 3

*******************  create basic dimensions for network  *********************


In [4]:
    print ('*******************  check the environment  *********************')
    print ("Pytorch version {}".format(torch.__version__))
    # check if CUDA is available
    use_cuda = torch.cuda.is_available()
    print("Is CUDA available? {}".format(use_cuda))
    # CUDA settings torch.__version__ must > 0.4
    # !!! This is important for the model!!! The first option is gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

*******************  check the environment  *********************
Pytorch version 1.1.0
Is CUDA available? True


In [5]:
    print ('*******************  load exsited ConvLSTM model  *********************')
    #model_init = torch.load(os.path.join(init_path, 'convlstm_era_sic_t2m_slp_z500_z850_uv10m_sflux_oras_ohc_Barents_hl_3_kernel_3_lr_0.005_epoch_1500_validAll.pkl'))
    model_init = torch.load(os.path.join(init_path, 'convlstm_era_sic_oras_ohc_Barents_hl_3_kernel_3_lr_0.005_epoch_1500_validAll.pkl'))
    print(model_init)

*******************  load exsited ConvLSTM model  *********************
ConvLSTM(
  (cell0): ConvLSTMCell(
    (Wxi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxc): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whc): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxo): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Who): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (cell1): ConvLSTMCell(
    (Wxi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1,

In [6]:
    print ('*******************  extract the weight from ConvLSTM  *********************')
    weight_model_init = {}
    # the weights are pytorch tensors
    for name, param in model_init.named_parameters():
        weight_model_init['{}'.format(name)] = param.data
        
    print ('*******************  check the weight from ConvLSTM  *********************')
    for i in weight_model_init:
        print('{}'.format(i))
        print('{}'.format(weight_model_init['{}'.format(i)]))
        print ("=========================")

*******************  extract the weight from ConvLSTM  *********************
*******************  check the weight from ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 3.6637e-01,  3.7026e-01,  3.4332e-01],
          [ 1.2342e-01,  1.0558e-01,  3.4235e-01],
          [ 2.5618e-01,  2.4329e-01,  1.2306e-01]],

         [[ 1.8853e-01,  4.7965e-02,  2.1323e-01],
          [ 4.6747e-04,  3.3257e-01,  1.2348e-01],
          [ 1.1532e-01,  1.0493e-01,  2.2473e-01]],

         [[ 1.3508e-01,  9.1303e-02,  1.8632e-01],
          [-6.3895e-02, -7.8429e-02,  4.4595e-02],
          [ 3.2660e-02, -1.5428e-02,  9.6294e-02]]],


        [[[ 1.7214e-01, -1.0985e-01,  1.2324e-01],
          [ 7.2119e-01,  1.4341e+00,  1.7877e+00],
          [ 1.9054e-01,  2.7632e-01,  5.5363e-01]],

         [[ 1.7301e-02,  3.0248e-01,  1.4761e-01],
          [ 3.8295e-01,  3.5788e-01,  9.7588e-02],
          [ 9.2612e-02,  1.0791e-01, -2.1884e-02]],

         [[ 2.7579e-02,  1.1252e-01,  3.0359e-02],
   

          [ 0.0618,  0.1370,  0.5484]]]], device='cuda:0')
cell0.Wxo.bias
tensor([-0.4694,  0.0403, -0.0357], device='cuda:0')
cell0.Who.weight
tensor([[[[ 0.2938,  0.2179,  0.2076],
          [ 0.1056, -0.0126,  0.2414],
          [ 0.3971,  0.0401,  0.0099]],

         [[-0.0922, -0.1542, -0.3577],
          [-0.0744, -0.1409, -0.1883],
          [-0.2443, -0.1836, -0.3893]],

         [[ 0.1526, -0.1654, -0.2321],
          [-0.5730, -0.3906, -0.1332],
          [ 0.0295, -0.0888, -0.3586]]],


        [[[ 0.0152, -0.0908,  0.1090],
          [ 0.2325, -0.0866, -0.0329],
          [ 0.4708,  0.2141,  0.0748]],

         [[ 0.2736,  0.0150,  0.2506],
          [ 0.1071,  0.2505, -0.0478],
          [ 0.1129, -0.0292,  0.0012]],

         [[ 0.5386, -0.1399,  0.0890],
          [-0.2497, -0.1919, -0.1980],
          [ 0.4405,  0.0627, -0.3187]]],


        [[[-0.1424, -0.0810, -0.3250],
          [-0.0192,  0.0675, -0.2480],
          [-0.2854, -0.0468, -0.2001]],

         [[ 0.4286,

          [-0.1496, -0.0849, -0.0688]]]], device='cuda:0')
cell1.Wxo.weight
tensor([[[[-0.3563, -0.3198, -0.2059],
          [-0.1494, -0.3098, -0.1616],
          [-0.4537, -0.5400, -0.3464]],

         [[ 0.5443,  0.3018,  0.5142],
          [ 0.4703,  0.5573,  0.3072],
          [ 0.2536,  0.0521,  0.0733]],

         [[ 0.2074,  0.4353,  0.2281],
          [ 0.5585,  0.3818,  0.4543],
          [ 0.3818,  0.2578,  0.3634]]],


        [[[ 0.0284, -0.1662,  0.0579],
          [ 0.0908, -0.1876, -0.0943],
          [-0.0628, -0.1806, -0.1277]],

         [[ 0.3661,  0.2070,  0.2064],
          [ 0.1869,  0.0273,  0.1441],
          [ 0.3176,  0.2798,  0.4512]],

         [[ 0.2271,  0.0730,  0.1791],
          [-0.0601,  0.3328,  0.2070],
          [ 0.0044, -0.0570,  0.1356]]],


        [[[-0.0311, -0.2925, -0.2805],
          [ 0.0210, -0.2440, -0.1489],
          [-0.0616, -0.0830, -0.0895]],

         [[ 0.1295,  0.2319, -0.0543],
          [-0.1890,  0.2150, -0.0096],
         

          [ 0.1985,  0.0769, -0.0263]]]], device='cuda:0')


In [7]:
    print ('***************  create new ConvLSTM model with chosen package *****************')
    model_convlstm = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size,
                                             weight_dict = weight_model_init).to(device)
    # save the ConvLSTM sample
    torch.save(model_convlstm.state_dict(), os.path.join(output_path,'convlstm_dlacs.pkl'))

***************  create new ConvLSTM model with chosen package *****************


In [8]:
    print ('*******************  check the weight from ConvLSTM  *********************')
    for name, param in model_convlstm.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 3.6637e-01,  3.7026e-01,  3.4332e-01],
          [ 1.2342e-01,  1.0558e-01,  3.4235e-01],
          [ 2.5618e-01,  2.4329e-01,  1.2306e-01]],

         [[ 1.8853e-01,  4.7965e-02,  2.1323e-01],
          [ 4.6747e-04,  3.3257e-01,  1.2348e-01],
          [ 1.1532e-01,  1.0493e-01,  2.2473e-01]],

         [[ 1.3508e-01,  9.1303e-02,  1.8632e-01],
          [-6.3895e-02, -7.8429e-02,  4.4595e-02],
          [ 3.2660e-02, -1.5428e-02,  9.6294e-02]]],


        [[[ 1.7214e-01, -1.0985e-01,  1.2324e-01],
          [ 7.2119e-01,  1.4341e+00,  1.7877e+00],
          [ 1.9054e-01,  2.7632e-01,  5.5363e-01]],

         [[ 1.7301e-02,  3.0248e-01,  1.4761e-01],
          [ 3.8295e-01,  3.5788e-01,  9.7588e-02],
          [ 9.2612e-02,  1.0791e-01, -2.1884e-02]],

         [[ 2.7579e-02,  1.1252e-01,  3.0359e-02],
          [-8.1348e-02,  4.0861e-02,  8.6488e-02],
          [-5.3958e-02,  8.03

          [ 0.0618,  0.1370,  0.5484]]]], device='cuda:0')
torch.Size([3, 3, 3, 3])
cell0.Wxo.bias
tensor([-0.4694,  0.0403, -0.0357], device='cuda:0')
torch.Size([3])
cell0.Who.weight
tensor([[[[ 0.2938,  0.2179,  0.2076],
          [ 0.1056, -0.0126,  0.2414],
          [ 0.3971,  0.0401,  0.0099]],

         [[-0.0922, -0.1542, -0.3577],
          [-0.0744, -0.1409, -0.1883],
          [-0.2443, -0.1836, -0.3893]],

         [[ 0.1526, -0.1654, -0.2321],
          [-0.5730, -0.3906, -0.1332],
          [ 0.0295, -0.0888, -0.3586]]],


        [[[ 0.0152, -0.0908,  0.1090],
          [ 0.2325, -0.0866, -0.0329],
          [ 0.4708,  0.2141,  0.0748]],

         [[ 0.2736,  0.0150,  0.2506],
          [ 0.1071,  0.2505, -0.0478],
          [ 0.1129, -0.0292,  0.0012]],

         [[ 0.5386, -0.1399,  0.0890],
          [-0.2497, -0.1919, -0.1980],
          [ 0.4405,  0.0627, -0.3187]]],


        [[[-0.1424, -0.0810, -0.3250],
          [-0.0192,  0.0675, -0.2480],
          [-0.2854,

          [-0.1496, -0.0849, -0.0688]]]], device='cuda:0')
torch.Size([3, 3, 3, 3])
cell1.Wxo.weight
tensor([[[[-0.3563, -0.3198, -0.2059],
          [-0.1494, -0.3098, -0.1616],
          [-0.4537, -0.5400, -0.3464]],

         [[ 0.5443,  0.3018,  0.5142],
          [ 0.4703,  0.5573,  0.3072],
          [ 0.2536,  0.0521,  0.0733]],

         [[ 0.2074,  0.4353,  0.2281],
          [ 0.5585,  0.3818,  0.4543],
          [ 0.3818,  0.2578,  0.3634]]],


        [[[ 0.0284, -0.1662,  0.0579],
          [ 0.0908, -0.1876, -0.0943],
          [-0.0628, -0.1806, -0.1277]],

         [[ 0.3661,  0.2070,  0.2064],
          [ 0.1869,  0.0273,  0.1441],
          [ 0.3176,  0.2798,  0.4512]],

         [[ 0.2271,  0.0730,  0.1791],
          [-0.0601,  0.3328,  0.2070],
          [ 0.0044, -0.0570,  0.1356]]],


        [[[-0.0311, -0.2925, -0.2805],
          [ 0.0210, -0.2440, -0.1489],
          [-0.0616, -0.0830, -0.0895]],

         [[ 0.1295,  0.2319, -0.0543],
          [-0.1890,  0.

tensor([ 0.0923, -0.0088], device='cuda:0')
torch.Size([2])
cell2.Who.weight
tensor([[[[-0.0089,  0.2559, -0.0267],
          [-0.0827, -0.2151,  0.3937],
          [-0.0746, -0.0604,  0.2563]],

         [[ 0.0398,  0.1071, -0.0255],
          [ 0.2014, -0.1520, -0.0306],
          [ 0.2865,  0.4267, -0.0932]]],


        [[[ 0.2775,  0.3157,  0.2511],
          [-0.0917,  0.2938,  0.3695],
          [-0.0490,  0.2909,  0.2252]],

         [[-0.1373, -0.1298,  0.1820],
          [ 0.0947,  0.2899,  0.0294],
          [ 0.1985,  0.0769, -0.0263]]]], device='cuda:0')
torch.Size([2, 2, 3, 3])


In [9]:
    print ('*******************  reload saved ConvLSTM model  *********************')
    # load model parameters
    model_reload = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    model_reload.load_state_dict(torch.load(os.path.join(output_path, 'convlstm_dlacs.pkl'),
                                map_location=device))
    # load entire model
    #model_reload = torch.load(os.path.join(output_path, 'sample_convlstm_lorenz84_0.pkl'))
    print(model_reload)

*******************  reload saved ConvLSTM model  *********************
ConvLSTM(
  (cell0): ConvLSTMCell(
    (Wxi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxc): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whc): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxo): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Who): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (cell1): ConvLSTMCell(
    (Wxi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Whi): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (Wxf): Conv2d(3, 3, kernel_size=(3, 3), stride=(1,

In [10]:
    print ('*******************  check the weight from reloaded ConvLSTM  *********************')
    for name, param in model_reload.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from reloaded ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 3.6637e-01,  3.7026e-01,  3.4332e-01],
          [ 1.2342e-01,  1.0558e-01,  3.4235e-01],
          [ 2.5618e-01,  2.4329e-01,  1.2306e-01]],

         [[ 1.8853e-01,  4.7965e-02,  2.1323e-01],
          [ 4.6747e-04,  3.3257e-01,  1.2348e-01],
          [ 1.1532e-01,  1.0493e-01,  2.2473e-01]],

         [[ 1.3508e-01,  9.1303e-02,  1.8632e-01],
          [-6.3895e-02, -7.8429e-02,  4.4595e-02],
          [ 3.2660e-02, -1.5428e-02,  9.6294e-02]]],


        [[[ 1.7214e-01, -1.0985e-01,  1.2324e-01],
          [ 7.2119e-01,  1.4341e+00,  1.7877e+00],
          [ 1.9054e-01,  2.7632e-01,  5.5363e-01]],

         [[ 1.7301e-02,  3.0248e-01,  1.4761e-01],
          [ 3.8295e-01,  3.5788e-01,  9.7588e-02],
          [ 9.2612e-02,  1.0791e-01, -2.1884e-02]],

         [[ 2.7579e-02,  1.1252e-01,  3.0359e-02],
          [-8.1348e-02,  4.0861e-02,  8.6488e-02],
          [-5.3958e-

          [ 0.0618,  0.1370,  0.5484]]]], device='cuda:0')
torch.Size([3, 3, 3, 3])
cell0.Wxo.bias
tensor([-0.4694,  0.0403, -0.0357], device='cuda:0')
torch.Size([3])
cell0.Who.weight
tensor([[[[ 0.2938,  0.2179,  0.2076],
          [ 0.1056, -0.0126,  0.2414],
          [ 0.3971,  0.0401,  0.0099]],

         [[-0.0922, -0.1542, -0.3577],
          [-0.0744, -0.1409, -0.1883],
          [-0.2443, -0.1836, -0.3893]],

         [[ 0.1526, -0.1654, -0.2321],
          [-0.5730, -0.3906, -0.1332],
          [ 0.0295, -0.0888, -0.3586]]],


        [[[ 0.0152, -0.0908,  0.1090],
          [ 0.2325, -0.0866, -0.0329],
          [ 0.4708,  0.2141,  0.0748]],

         [[ 0.2736,  0.0150,  0.2506],
          [ 0.1071,  0.2505, -0.0478],
          [ 0.1129, -0.0292,  0.0012]],

         [[ 0.5386, -0.1399,  0.0890],
          [-0.2497, -0.1919, -0.1980],
          [ 0.4405,  0.0627, -0.3187]]],


        [[[-0.1424, -0.0810, -0.3250],
          [-0.0192,  0.0675, -0.2480],
          [-0.2854,

          [-0.1496, -0.0849, -0.0688]]]], device='cuda:0')
torch.Size([3, 3, 3, 3])
cell1.Wxo.weight
tensor([[[[-0.3563, -0.3198, -0.2059],
          [-0.1494, -0.3098, -0.1616],
          [-0.4537, -0.5400, -0.3464]],

         [[ 0.5443,  0.3018,  0.5142],
          [ 0.4703,  0.5573,  0.3072],
          [ 0.2536,  0.0521,  0.0733]],

         [[ 0.2074,  0.4353,  0.2281],
          [ 0.5585,  0.3818,  0.4543],
          [ 0.3818,  0.2578,  0.3634]]],


        [[[ 0.0284, -0.1662,  0.0579],
          [ 0.0908, -0.1876, -0.0943],
          [-0.0628, -0.1806, -0.1277]],

         [[ 0.3661,  0.2070,  0.2064],
          [ 0.1869,  0.0273,  0.1441],
          [ 0.3176,  0.2798,  0.4512]],

         [[ 0.2271,  0.0730,  0.1791],
          [-0.0601,  0.3328,  0.2070],
          [ 0.0044, -0.0570,  0.1356]]],


        [[[-0.0311, -0.2925, -0.2805],
          [ 0.0210, -0.2440, -0.1489],
          [-0.0616, -0.0830, -0.0895]],

         [[ 0.1295,  0.2319, -0.0543],
          [-0.1890,  0.

tensor([ 0.0923, -0.0088], device='cuda:0')
torch.Size([2])
cell2.Who.weight
tensor([[[[-0.0089,  0.2559, -0.0267],
          [-0.0827, -0.2151,  0.3937],
          [-0.0746, -0.0604,  0.2563]],

         [[ 0.0398,  0.1071, -0.0255],
          [ 0.2014, -0.1520, -0.0306],
          [ 0.2865,  0.4267, -0.0932]]],


        [[[ 0.2775,  0.3157,  0.2511],
          [-0.0917,  0.2938,  0.3695],
          [-0.0490,  0.2909,  0.2252]],

         [[-0.1373, -0.1298,  0.1820],
          [ 0.0947,  0.2899,  0.0294],
          [ 0.1985,  0.0769, -0.0263]]]], device='cuda:0')
torch.Size([2, 2, 3, 3])
